Prediction for output : Max deformation

In [1]:
#necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


In [21]:
#data loading (change it to whatever u have as path)
df = pd.read_csv("C:/Users/asmae/Desktop/augmented2_data_simulation.csv")

#Target : Max deformation
X = df[['Vitesse', 'Temps', 'Profondeur']].values
y = df['Max contrainte'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [137]:


#creating a pipeline that first scales the data then applies GPR
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('gpr', GaussianProcessRegressor(normalize_y=True, n_restarts_optimizer=5))
])


In [138]:
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split


In [139]:
param_grid = {
    'gpr__kernel': [C(1.0, (1e-9, 1e9)) * RBF(length_scale=l) + WhiteKernel(noise_level=1e-5) for l in np.logspace(-3, 3, 7)]
}


In [140]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('gpr',
                                        GaussianProcessRegressor(n_restarts_optimizer=5,
                                                                 normalize_y=True))]),
             param_grid={'gpr__kernel': [1**2 * RBF(length_scale=0.001) + WhiteKernel(noise_level=1e-05),
                                         1**2 * RBF(length_scale=0.01) + WhiteKernel(noise_level=1e-05),
                                         1**2 * RBF(length_scale=0.1) + WhiteKernel(noise_level=1e-05),
                                         1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1e-05),
                                         1**2 * RBF(length_scale=10) + WhiteKernel(noise_level=1e-05),
                                         1**2 * RBF(length_scale=100) + WhiteKernel(noise_level=1e-05),
                                         1**2 * RBF(length_scale=1e+03) + WhiteKernel(noise_level=1e-05)]},
             scoring='neg_mean_squared_error')

In [141]:
# Predict on the test set
y_pred = grid_search.predict(X_test)



testing


In [151]:
# Fixed values for speed and depth ( vitesse - profondeur)
vitesse_new = 40
profondeur_new = 2

#range of time points at which we want to predict the outputs
time_points = np.array([0.0004,
0.00045,
0.0005,
0.00055,
0.0006,
0.00065,
0.0007,
0.00075,
0.0008,
0.00085,
0.0009,
])


In [152]:
#new input data 
new_data = np.column_stack((np.full(time_points.shape, vitesse_new),
                            time_points,
                            np.full(time_points.shape, profondeur_new)))


In [153]:
new_predictions =  grid_search.predict(new_data)


for time, pred in zip(time_points, new_predictions):
    print(f"Temps: {time}, Max contrainte: {pred:.20f}")

Temps: 0.0004, Max contrainte: 91699934.98707520961761474609
Temps: 0.00045, Max contrainte: 83724978.71436759829521179199
Temps: 0.0005, Max contrainte: 75633975.84523926675319671631
Temps: 0.00055, Max contrainte: 69956464.30161654949188232422
Temps: 0.0006, Max contrainte: 66937268.11522624641656875610
Temps: 0.00065, Max contrainte: 65641981.36145406961441040039
Temps: 0.0007, Max contrainte: 65177765.77567918598651885986
Temps: 0.00075, Max contrainte: 65041800.24032181501388549805
Temps: 0.0008, Max contrainte: 65013558.55302577465772628784
Temps: 0.00085, Max contrainte: 65012023.24894440174102783203
Temps: 0.0009, Max contrainte: 65013818.80125474184751510620
